## Training 

In [68]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import pickle
import sklearn

In [39]:
#Loading data and embeddings
label_embeddings = pickle.load(open("Data/Embeddings/CIFAR/CIFAR_100_label_to_embedding_google_news.pk", "rb"))
train_x = np.load("vgg_16/train_x.npy").reshape((50000,8192))
train_y = np.load("vgg_16/train_y.npy")
test_x = np.load("vgg_16/test_x.npy").reshape((10000,8192))
test_y = np.load("vgg_16/test_y.npy")


(10000, 1)


In [184]:
#Convert train/test labels to embeddings
train_y_embeddings = np.zeros((50000, 300))
test_y_embeddings = np.zeros((10000, 300))

for i in range(train_y.shape[0]):
    train_y_embeddings[i] = label_embeddings[int(train_y[i][0])]

for i in range(test_y.shape[0]):
    test_y_embeddings[i] = label_embeddings[int(test_y[i][0])]

In [185]:
model = Sequential()
model.add(Dense(400, input_dim=8192, activation='relu'))
model.add(Dense(400, input_dim=400, activation='relu'))
model.add(Dense(300, input_dim=400, activation='relu'))

In [186]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [187]:
model.fit(train_x, train_y_embeddings, epochs=1, batch_size=64)

Epoch 1/1
50000/50000 [==============================] - 69s 1ms/step - loss: 0.0353 - acc: 0.0885


## Testing

In [192]:
from sklearn.metrics.pairwise import cosine_similarity

In [193]:
pred_test_y = model.predict(test_x)

label_embeddings
label_embeddings_arr = np.zeros((100, 300))
for i in range(100):
    label_embeddings_arr[i] = label_embeddings[i]

In [194]:
#Predictions
label_predictions = cosine_similarity(pred_test_y, label_embeddings_arr).argmax(axis=1)


In [195]:
#Accuracy
acc = np.sum((np.squeeze(test_y) == label_predictions)) / float(test_y.shape[0])
print("Accuracy: ", acc)

('Accuracy: ', 0.058299999999999998)


In [196]:
train_y_embeddings

array([[-0.12304688,  0.28710938, -0.27734375, ...,  0.22558594,
         0.25390625,  0.08544922],
       [ 0.15625   , -0.21875   , -0.28125   , ...,  0.00405884,
        -0.21289062,  0.29882812],
       [ 0.375     , -0.01257324, -0.04418945, ...,  0.18261719,
        -0.04345703, -0.15722656],
       ..., 
       [ 0.18847656,  0.03930664, -0.234375  , ...,  0.18945312,
        -0.17089844,  0.08251953],
       [ 0.32617188,  0.13085938,  0.03466797, ..., -0.30273438,
        -0.08007812,  0.02099609],
       [ 0.05322266,  0.23730469, -0.21582031, ...,  0.24804688,
        -0.02331543,  0.03710938]])